In [53]:
from __future__ import division      #除数可以显示为float

import jqdata               #导入聚宽函数库
from six import StringIO    #使用聚宽readfile函数
import numpy as np
import pandas as pd
import talib

import time                 #使用time stamp
import datetime             

import matplotlib.pyplot as plt

In [54]:
# 确定全局变量
win_num = 1.015       #获胜的比例
lose_num = 0.985      #失败的比例
even_num = 5          #等待的天数

导入模块

筛选股票，生成股票列表

In [55]:
stock_to_choose = get_fundamentals(query(
        valuation.code, valuation.pe_ratio, 
        valuation.pb_ratio,valuation.market_cap, 
        indicator.eps, indicator.inc_net_profit_annual
    ).filter(
        valuation.pe_ratio > 2
    ).order_by(
        valuation.market_cap.desc()
    ), date=None)

stock_list_all = list(stock_to_choose['code'])


#用于测试，分别注解不同的语句，控制股票序列大小
# stock_list_all = stock_list_all[:]
# stock_list_all = ['002558.XSHE', '600362.XSHG', '600516.XSHG', '603260.XSHG', '000898.XSHE', '002202.XSHE', '600271.XSHG', '600066.XSHG', '600377.XSHG', '002602.XSHE']
stock_list_all = ['000651.XSHE','000858.XSHE']
stock_list_all = get_index_stocks('000300.XSHG')
# stock_list_all = ['000651.XSHE']
# print stock_list_all

[u'000001.XSHE', u'000002.XSHE', u'000060.XSHE', u'000063.XSHE', u'000069.XSHE', u'000100.XSHE', u'000157.XSHE', u'000166.XSHE', u'000333.XSHE', u'000338.XSHE', u'000402.XSHE', u'000413.XSHE', u'000415.XSHE', u'000423.XSHE', u'000425.XSHE', u'000503.XSHE', u'000538.XSHE', u'000540.XSHE', u'000559.XSHE', u'000568.XSHE', u'000623.XSHE', u'000625.XSHE', u'000627.XSHE', u'000630.XSHE', u'000651.XSHE', u'000671.XSHE', u'000709.XSHE', u'000723.XSHE', u'000725.XSHE', u'000728.XSHE', u'000768.XSHE', u'000776.XSHE', u'000783.XSHE', u'000786.XSHE', u'000792.XSHE', u'000826.XSHE', u'000839.XSHE', u'000858.XSHE', u'000876.XSHE', u'000895.XSHE', u'000898.XSHE', u'000938.XSHE', u'000959.XSHE', u'000961.XSHE', u'000963.XSHE', u'000983.XSHE', u'001965.XSHE', u'001979.XSHE', u'002007.XSHE', u'002008.XSHE', u'002024.XSHE', u'002027.XSHE', u'002044.XSHE', u'002050.XSHE', u'002065.XSHE', u'002074.XSHE', u'002081.XSHE', u'002085.XSHE', u'002142.XSHE', u'002146.XSHE', u'002153.XSHE', u'002202.XSHE', u'00223

生成价格字典
为提升操作性，价格 = 0.5*收盘价 + 0.5*开盘价

In [56]:
price = {}          #{股票：[价格1，价格2...]}

for stock_name in set(stock_list_all):
    stock_price = get_price(stock_name, count = 2000, end_date='2018-5-1', frequency='1d', 
                            fields=['open', 'close'], fq='none')

    stock_price['avg'] = stock_price['open']*0.5 + stock_price['close']*0.5
   
    price[stock_name] = list(stock_price['avg'])

# print price

按照价格生成胜率列表
胜率标准：
1. 5天内超过1.015记为“win”
2. 5天内低于0.985记为“lose”
3. 5天内未变化价位“even”

In [57]:
# print price

win_rate = {}         #{股票编码：[win，lose，even...]}
# 按照每个股票为索引进行数据处理
for stock_name in price.keys():
    win_rate[stock_name] = []
    
    # 按照序列逐个处理每个价格列表
    # 注意：为了避免最后下标越界，所以只对[：-5]的序列做判断
    
    for i in range(0,int(len(price[stock_name]) - even_num)):
        
        # 按照5天作为周期，逐个对比，若符合要求就退出for循环
        for day_count in range(1,even_num + 1):
            if price[stock_name][i+day_count]:
                pass
            
            
            if price[stock_name][i+day_count] / price[stock_name][i] >= win_num:
                win_rate[stock_name].append("win")
                break
            elif price[stock_name][i+day_count] / price[stock_name][i] <= lose_num:                
                win_rate[stock_name].append("lose")
                break
            else:
                win_rate[stock_name].append("even")
                break
    
#     print len(price[stock_name])    
#     print int(len(price[stock_name]) - even_num)
#     print stock_name
#     print len(win_rate[stock_name])
#     print win_rate[stock_name]

# print win_rate

开始计算MACD金叉

In [58]:
# print price

stock_macd = {}

fastp = 5
slowp = 10
sigp = 5

for stock_name in price.keys():

    stock_macd[stock_name] = []
    if len(price[stock_name]) > 50:

        dif, dea, macd = talib.MACD(np.array(price[stock_name]), fastperiod=fastp, slowperiod=slowp, signalperiod=sigp)
  
        macd_gold = np.where((macd[:-1] < 0) & (macd[1:] > 0))[0]  + 1
    
        # 为了防止太靠近当前，防止越界，所以剔除了最后面几天的金叉
        # 注意：次序列的金叉指的是监测到当天发生金叉，注意对比时，需要对比后一天的价格数据，保证可操作性
        # 注意：目前已经加1，后续不用再操作
        macd_gold = [elem for elem in macd_gold if elem < (len(macd) - even_num) ]
        
        
        stock_macd[stock_name] = macd_gold

# print stock_macd
        

开始计算价格成功率与MACD金叉成功率的关联性

In [59]:
# print win_rate
# print stock_macd

stock_winrate_result = {}

for stock_name in win_rate.keys():

    stock_winrate_result[stock_name] = map(lambda x: win_rate[stock_name][x], stock_macd[stock_name])


开始分股票计算胜率
按照目前初步确定的情况，MACD金叉并没有明显的取得胜率

但是就样本数据来看，输的次数远多于赢得次数

In [73]:
# print stock_winrate_result
possibility = {}

for stock_name in stock_winrate_result.keys():
    if len(stock_winrate_result[stock_name]) != 0:
        possibility[stock_name] = [sum(list(np.array(stock_winrate_result[stock_name])== 'win')), \
                                 sum(list(np.array(stock_winrate_result[stock_name])== 'lose')), \
                                 sum(list(np.array(stock_winrate_result[stock_name])== 'even'))]



[]


{u'600438.XSHG': [22, 8, 37], u'000630.XSHE': [9, 4, 40], u'600030.XSHG': [14, 3, 40], u'601216.XSHG': [10, 6, 41], u'601857.XSHG': [7, 2, 54], u'601919.XSHG': [12, 1, 39], u'601108.XSHG': [0, 0, 2], u'600111.XSHG': [16, 5, 39], u'601211.XSHG': [2, 0, 15], u'600340.XSHG': [25, 10, 42], u'600741.XSHG': [14, 2, 41], u'600100.XSHG': [14, 4, 44], u'000876.XSHE': [12, 5, 46], u'002460.XSHE': [21, 7, 28], u'002602.XSHE': [9, 5, 26], u'601788.XSHG': [14, 3, 45], u'002450.XSHE': [18, 6, 31], u'002065.XSHE': [20, 4, 37], u'300033.XSHE': [16, 15, 31], u'600011.XSHG': [15, 3, 52], u'601866.XSHG': [10, 4, 42], u'000783.XSHE': [15, 4, 46], u'000540.XSHE': [14, 2, 37], u'002202.XSHE': [15, 4, 46], u'300059.XSHE': [23, 8, 28], u'600066.XSHG': [18, 2, 50], u'601088.XSHG': [8, 6, 51], u'601991.XSHG': [9, 5, 56], u'600900.XSHG': [5, 0, 54], u'002008.XSHE': [18, 9, 35], u'600498.XSHG': [15, 5, 43], u'002572.XSHE': [14, 6, 40], u'000425.XSHE': [14, 2, 44], u'000961.XSHE': [22, 8, 35], u'000069.XSHE': [24,

计算总胜率

In [74]:
win_count = 0
lose_count = 0
even_count = 0

for stock_name in possibility.keys():
    win_count += possibility[stock_name][0]
    lose_count += possibility[stock_name][1]
    even_count += possibility[stock_name][2]
    
print win_count
print lose_count
print even_count


3913
1293
11058


开始研究MACD策略，需要确定：
- Diff大于0，是否意味着上涨的概率高于下跌的概率

In [61]:
stock_macd_diff = {}

fastp = 5
slowp = 10
sigp = 5

for stock_name in price.keys():

    stock_macd_diff[stock_name] = []
    if len(price[stock_name]) > 50:

        dif, dea, macd = talib.MACD(np.array(price[stock_name]), fastperiod=fastp, slowperiod=slowp, signalperiod=sigp)

        stock_macd_diff[stock_name] = (np.array(dif) > 0)
#     print len(stock_macd_diff[stock_name])


将取得的diff结果与下一天的胜率比较，看是否有相关性
注意：是下一天

按照目前的验证结果，相关性较低，相关性如果较高，需要大于0.8

In [63]:
# print stock_macd_diff
# print win_rate

for stock_name in stock_macd_diff.keys():
  
    # winrate 序列剔除了最后5天（even_num）
    # stock_macd_diff是完整的数据，但是需要保持预测性的话，需要移动一位，前1天的diff与后1天的winrate相比
    # 同时需要考虑diff前几位是不确定的，舍弃前面的30位
    # 所以最终应为：stock_macd_diff[stock_name]中[29:-6]与win_rate的[30:]相比
    #     print stock_macd_diff[stock_name][29:-(even_num+1)]
    #     print len(stock_macd_diff[stock_name][29:-(even_num+1)])
    #     print win_rate[stock_name][30:]
    #     print len(win_rate[stock_name][30:])
    #     print np.array(stock_macd_diff[stock_name][29:-(even_num+1)])
    #     print np.array(win_rate[stock_name][30:]) !="lose"
    
    cor_array = np.array([np.array(stock_macd_diff[stock_name][29:-(even_num+1)]),np.array(win_rate[stock_name][30:]) !="lose"])
    cor_pd = pd.DataFrame(cor_array.T, columns=['A', 'B'])
    print cor_pd.A.corr(cor_pd.B)

0.0341437108878
0.0212998650476
-0.0303839935427
-0.08893165961
-0.0293960009108
0.0314444915053
-0.309617541584
-0.0230037280351
-0.140363297044
-0.0725891257278
0.0524466118305
0.00477448893264
-0.00171383396095
-0.0555695692825
0.0132539015817
-0.0407250517041
-0.0708992377222
0.00553471451939
-0.0363413382329
-0.0286252822514
0.00400701132458
-0.0545130263701
-0.0553822997874
-0.0269990982872
-0.0568135007936
-0.0118610607523
0.0110066792536
-0.0437415883464
0.00247711569279
-0.0118183361691
-0.0130048106041
-0.0926517163281
0.0611459293809
0.00826081689517
-0.0512239647179
-0.0311763109859
-0.228443812752
0.0344572076722
0.0331525251886
-0.180829912776
-0.0619801341331
-0.123218396842
-0.0871869089664
-0.00587088895233
-0.0279923515137
-0.0105369744558
-0.198701289048
-0.106959849413
-0.204560474527
-0.403915231731
-0.0143526653741
-0.0257415224474
-0.140228513189
0.0279058556053
0.00647360043055
-0.0291905821986
-0.0445290222294
0.0157616386611
-0.0186564816917
0.00830494924185
-

开始研究均线侧率，需要确定：
- 在5日均线在10日均线以上，是否意味着上涨的概率高于下跌的概率

相关性依然不高，问题出在哪里？需要进一步再观察

In [64]:
for stock_name in price.keys():
    
    EMA_gap = np.array([])
 
    EMA_gap = talib.EMA(np.array(price[stock_name]), timeperiod=6) - talib.EMA(np.array(price[stock_name]), timeperiod=12)   
#     print EMA_gap
#     print EMA_gap >0
    
#     print list(EMA_gap)
    EMA_gap[29:-(even_num+1)]
    cor_array = np.array([EMA_gap[29:-(even_num+1)],np.array(win_rate[stock_name][30:]) !="lose"])
    cor_pd = pd.DataFrame(cor_array.T, columns=['A', 'B'])
    print cor_pd.A.corr(cor_pd.B)

-0.0145669743215
0.0323615177297
0.0570694943162
-0.0101356974955
-0.0583417612662
0.0306458746702
-0.0320036721872
-0.00992386598592
-0.0334227848758
-0.0495457129205
0.0341726411404
0.0273992571309
-0.0416185339139
-0.0735694232204
-0.00295958769348
-0.00509273754035
0.00239494549742
0.0694896190747
-0.0305358686144
-0.0319746951899
0.0139750319177
-0.058611638414
0.0069443539251
-0.0163114165233
-0.0560880937683
0.00353148228818
-0.0257449677696
-0.0721723138197
0.00559482806429
0.000351658271388
0.011953182942
-0.0194725592555
0.0541012263668
0.0047093389241
-0.00945677409804
-0.0406335353045
-0.0471573017587
-0.0448141232595
0.0106504256825
-0.0865595098942
-0.0427565848149
-0.0208402825688
-0.19791847639
-0.0573979941455
-0.0181686928107
0.0494097969651
-0.0133485037944
-0.0678865614743
0.117836527611
-0.00995751605054
-0.126685836214
-0.0187868766034
0.0122597120822
-0.0361507860713
-0.041328930675
-0.0132654820102
0.0157093768326
-0.0640316305684
0.0240352085978
0.0073136355994

研究均线金叉对指标的影响

In [65]:
stock_ema = {}

for stock_name in price.keys():
    
    EMA_gap = np.array([])
 
    EMA_gap = talib.EMA(np.array(price[stock_name]), timeperiod=6) - talib.EMA(np.array(price[stock_name]), timeperiod=12)   

    EMA_gold = np.where((EMA_gap[:-1] < 0) & (EMA_gap[1:] > 0))[0]  + 1
    EMA_gold = [elem for elem in EMA_gold if elem < (len(EMA_gap) - even_num)]
    
    stock_ema[stock_name] = EMA_gold
    
#                 # 注意：目前已经加1，后续不用再操作
#         macd_gold = [elem for elem in macd_gold if elem < (len(macd) - even_num) ]

# print stock_ema
#         stock_macd[stock_name] = macd_gold

In [66]:
# print win_rate
# print stock_macd

stock_winrate_result = {}

for stock_name in win_rate.keys():
    stock_winrate_result[stock_name] = map(lambda x: win_rate[stock_name][x], stock_ema[stock_name])
    
# print stock_winrate_result

# print stock_winrate_result

for stock_name in stock_winrate_result.keys():
    print sum(list(np.array(stock_winrate_result[stock_name])== 'win'))
    print sum(list(np.array(stock_winrate_result[stock_name])== 'lose'))    
    print sum(list(np.array(stock_winrate_result[stock_name])== 'even'))    
    
    
#     print len(stock_winrate_result[stock_name])
#     print sum(np.array(stock_winrate_result[stock_name]) == "win")

9
4
40
13
15
41
10
6
41
7
2
54
12
1
39
0
0
2
16
5
39
16
2
46
25
10
42
14
2
41
14
4
44
12
5
46
21
7
28
9
5
26
14
3
45
18
6
31
3
1
7
16
15
31
15
3
52
10
4
42
15
4
46
14
2
37
15
4
46
23
8
28
18
2
50
8
6
51
5
0
54
18
9
35
15
5
43
14
6
40
14
6
39
14
2
44
22
8
35
24
5
42
13
8
42
3
3
17
16
0
44
13
2
47
0
1
1
24
8
39
16
2
48
2
1
10
19
9
30
20
10
35
8
5
34
13
4
43
6
0
16
1
0
6
6
2
11
19
1
38


/opt/conda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:14: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


TypeError: 'bool' object is not iterable